In [1]:
!pip install kaggle # 케글 데이터 설치

# kaggle.json 파일 코렙드라이브로 마운팅하기
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jonum123","key":"c1ca2e09bb982a2d2b88a0f7f333377a"}'}

In [2]:
# 케글 파일 만들기
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 이 일어나지 않도록
!chmod 600 ~/.kaggle/kaggle.json

# 케글 제이슨 파일 제대로 설치 됬는지 확인
!ls -lha kaggle.json # kaggle.json 이렇게 뜸

-rw-r--r-- 1 root root 64 Jun 18 12:23 kaggle.json


In [3]:
! kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

'''다운로드 성공시 표시됨
Downloading gtsrb-german-traffic-sign.zip to /content
 97% 594M/612M [00:03<00:00, 221MB/s]
100% 612M/612M [00:03<00:00, 207MB/s]
'''

!unzip gtsrb-german-traffic-sign.zip # zip 파일 풀기

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/5/00005_00053_00010.png  
  inflating: train/5/00005_00053_00011.png  
  inflating: train/5/00005_00053_00012.png  
  inflating: train/5/00005_00053_00013.png  
  inflating: train/5/00005_00053_00014.png  
  inflating: train/5/00005_00053_00015.png  
  inflating: train/5/00005_00053_00016.png  
  inflating: train/5/00005_00053_00017.png  
  inflating: train/5/00005_00053_00018.png  
  inflating: train/5/00005_00053_00019.png  
  inflating: train/5/00005_00053_00020.png  
  inflating: train/5/00005_00053_00021.png  
  inflating: train/5/00005_00053_00022.png  
  inflating: train/5/00005_00053_00023.png  
  inflating: train/5/00005_00053_00024.png  
  inflating: train/5/00005_00053_00025.png  
  inflating: train/5/00005_00053_00026.png  
  inflating: train/5/00005_00053_00027.png  
  inflating: train/5/00005_00053_00028.png  
  inflating: train/5/00005_00053_00029.png  
  inflating: train/5/00005_00054_00000.png  
  inflating: train/

In [5]:
import os

# 현재 디렉토리의 파일 목록을 출력
print(os.listdir('.'))
print(os.listdir('/content'))

['.config', 'CNN_model1.pt', 'train', 'Train.csv', 'meta', 'Test.csv', 'Test', 'CNN_model.pth', 'test', 'Train', 'Meta', 'gtsrb-german-traffic-sign.zip', 'Meta.csv', 'kaggle.json', 'sample_data']
['.config', 'CNN_model1.pt', 'train', 'Train.csv', 'meta', 'Test.csv', 'Test', 'CNN_model.pth', 'test', 'Train', 'Meta', 'gtsrb-german-traffic-sign.zip', 'Meta.csv', 'kaggle.json', 'sample_data']


In [6]:
import cv2
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


device = torch.device("cuda"if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv1_ = nn.Conv2d(32, 32, kernel_size=3, padding=1)

        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv2_ = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3_ = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4_ = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.bn4 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout(0.5)

        self.activate = F.elu

        self.fc1 = nn.Linear(256 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 43)

    def forward(self, x):
        x = self.activate(self.bn1(self.conv1(x)))
        x = self.activate(self.conv1_(x))
        x = self.activate(self.conv1_(x))
        x = self.pool(x)

        x = self.activate(self.bn2(self.conv2(x)))
        x = self.activate(self.conv2_(x))
        x = self.activate(self.conv2_(x))
        x = self.pool(x)

        x = self.activate(self.bn3(self.conv3(x)))
        x = self.activate(self.conv3_(x))
        x = self.activate(self.conv3_(x))
        x = self.pool(x)

        x = self.activate(self.bn4(self.conv4(x)))
        x = self.activate(self.conv4_(x))
        x = self.activate(self.conv4_(x))
        x = self.pool(x)

        x = x.reshape(-1, 256 * 2 * 2)
        x = self.dropout(self.activate(self.fc1(x)))
        x = self.fc2(x)
        return x

In [8]:
####################
##### 성환씨모델 ######
####################

modelSH_path = './CNN_model.pth'
modelSH = torch.load(modelSH_path)
modelSH.eval()

print(modelSH)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): M

In [9]:
dataset_root = './'

def resize_image(image, size=(32,32)):
    return cv2.resize(image, size)

def normalize_image(image):
    return image / 255.0

def one_hot_encode(labels):
    encoder = OneHotEncoder(sparse_output=False)
    labels = np.array(labels).reshape(-1, 1)
    encoder.fit(labels)
    return encoder.transform(labels)

def load_data(folder):
    images = []
    labels = []

    train_df = pd.read_csv(os.path.join(dataset_root, folder))

    for index, row in train_df.iterrows():
        image_file = row['Path']
        image_path = os.path.join(dataset_root, image_file)

        image = cv2.imread(image_path)
        image = resize_image(image)
        image = normalize_image(image)

        images.append(image)
        labels.append(row['ClassId'])

    images = np.array(images)
    labels = np.array(labels)

    labels = one_hot_encode(labels)

    return images, labels

def apply_transforms(images, transform):
    res = []
    for image in images:
        img = transform(image)
        res.append(img.numpy())
    return np.array(res)

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

test_images, test_labels = load_data('Test.csv')

X_test = test_images
y_test = test_labels

X_test = apply_transforms(X_test, test_transforms)
X_test = np.array(X_test)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
modelSH.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = modelSH(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"ModelSH Test Accuracy: {correct / total}")

ModelSH Test Accuracy: 0.973396674584323


In [11]:
####################
##### 승우씨모델 ######
####################

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 43)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)
        x = x.reshape(-1, 128 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

modelSO_path = './CNN_model1.pt'
modelSO = torch.load(modelSO_path)
modelSO.eval()

# 모델 구조 확인
print(modelSO)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=43, bias=True)
)


In [12]:
modelSO.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = modelSO(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"ModelSO Test Accuracy: {correct / total}")

ModelSO Test Accuracy: 0.9505146476642914


In [13]:
# 앙상블 기법 적용
modelSO.eval()
modelSH.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)

        outputs = (outputsSO + outputsSH) / 2

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Ensemble Test Accuracy: {correct / total}")


Ensemble Test Accuracy: 0.9794932699920823


In [53]:
import torch

modelSO.eval()
modelSH.eval()
correct = 0
total = 0

# 가중치 설정
weightSH = 0.55
weightSO = 0.44

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)

        # 예측 확률에 가중치 적용하여 최종 예측 확률 계산
        outputs = (outputsSO * weightSO + outputsSH * weightSH)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.978701504354711


In [54]:
import torch

modelSO.eval()
modelSH.eval()
correct = 0
total = 0

# 가중치 설정
weightSH = 0.51
weightSO = 0.49

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)

        # 예측 확률에 가중치 적용하여 최종 예측 확률 계산
        outputs = (outputsSO * weightSO + outputsSH * weightSH)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.979334916864608


In [56]:
modelSO.eval()
modelSH.eval()
correct = 0
total = 0

# 가중치 설정
weightSH = 0.7
weightSO = 0.3

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)

        # 예측 확률에 가중치 적용하여 최종 예측 확률 계산
        outputs = (outputsSO * weightSO + outputsSH * weightSH)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9770387965162312


In [14]:
# 동성 모델 생성

# 라이브러리 import
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


## 학습 데이터 설정
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(32, scale = (0.7, 1.0)),
    transforms.ColorJitter(brightness = 0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor()
])

images_, labels_ = load_data('Train.csv')

images = images_.copy()
labels = labels_.copy()
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size = 0.2, random_state = 42)

X_train = apply_transforms(X_train, train_transforms)
X_val = apply_transforms(X_val, test_transforms)

X_train = np.array(X_train)
X_val = np.array(X_val)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [15]:
import torch
import torch.nn as nn
import torchvision.models as models

# ResNet-18
resnet = models.resnet18(pretrained=True)

# 첫 번째 합성곱 층, 맥스풀링 레이어 조정
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
resnet.maxpool = nn.Identity()

# fully-connected 레이어 조정
num_ftrs = resnet.fc.in_features  # 기존 레이어
resnet.fc = nn.Linear(num_ftrs, 43)  # 43개의 클래스로 변경

# 모델 구조 확인
print(resnet)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 142MB/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [18]:
model = resnet

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

num_epochs = 10

best_loss = 99
break_num = 5
break_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device) 
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    loss = running_loss/len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss}")
    if best_loss > loss:
        best_loss = loss
        break_counter = 0
    else:
        break_counter += 1
        if break_counter == break_num:
            print("Early Break")
            break;

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device) 
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Validation Accuracy: {correct / total}")

Epoch [1/10], Loss: 0.05725330385551123
Epoch [2/10], Loss: 0.037062612316031064
Epoch [3/10], Loss: 0.028307646605394338
Epoch [4/10], Loss: 0.023271056631842108
Epoch [5/10], Loss: 0.018376517511721438
Epoch [6/10], Loss: 0.03271580070588443
Epoch [7/10], Loss: 0.017016020219788815
Epoch [8/10], Loss: 0.018987498423052276
Epoch [9/10], Loss: 0.011657854961705339
Epoch [10/10], Loss: 0.019224658356408377
Validation Accuracy: 0.9957918898240244


In [20]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device) 
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Model Test Accuracy: {correct / total}")

Model Test Accuracy: 0.9726840855106889


In [21]:
import torch

modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        outputs = (outputsSO + outputsSH + outputsDS) / 3

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")

Weighted Ensemble Test Accuracy: 0.9844813935075217


In [22]:
# 모델 3개 앙상블 적용

import torch

modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

# 가중치 설정
weightSH = 0.4
weightSO = 0.3
weightDS = 0.4

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        # 예측 확률에 가중치 적용
        outputs = (outputsSO * weightSO + outputsSH * weightSH + outputsDS * weightDS)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9840855106888361


In [23]:
# 모델 3개 앙상블 적용

import torch

modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

# 가중치 설정
weightSH = 0.3
weightSO = 0.4
weightDS = 0.4

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        # 예측 확률에 가중치 적용
        outputs = (outputsSO * weightSO + outputsSH * weightSH + outputsDS * weightDS)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9847980997624703


In [25]:
# 모델 3개 앙상블 적용

import torch

modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

# 가중치 설정
weightSH = 0.3
weightSO = 0.35
weightDS = 0.35

with torch.no_grad():
    for inputs, labels in test_loader:
        # 두 모델의 예측 확률 계산
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        # 예측 확률에 가중치 적용
        outputs = (outputsSO * weightSO + outputsSH * weightSH + outputsDS * weightDS)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9849564528899446


In [29]:
modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

# 가중치 설정
weightSH = 0.3
weightSO = 0.37
weightDS = 0.33

with torch.no_grad():
    for inputs, labels in test_loader:
        
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        # 가중치 적용
        outputs = (outputsSO * weightSO + outputsSH * weightSH + outputsDS * weightDS)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9850356294536817


In [31]:
modelSO.eval()
modelSH.eval()
model.eval()

correct = 0
total = 0

# 가중치 설정
weightSH = 0.33
weightSO = 0.37
weightDS = 0.3

with torch.no_grad():
    for inputs, labels in test_loader:
        outputsSO = modelSO(inputs)
        outputsSH = modelSH(inputs)
        outputsDS = model(inputs)

        # 가중치 적용
        outputs = (outputsSO * weightSO + outputsSH * weightSH + outputsDS * weightDS)

        # 최종 예측 결정
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Weighted Ensemble Test Accuracy: {correct / total}")


Weighted Ensemble Test Accuracy: 0.9845605700712589
